In [ ]:
!pip install -q streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 105.2 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
import streamlit as st

"""
Streamlit app: FITS uploader & processo
Filename: streamlit_fits_uploader_and_processor.py

Features implemented (best-effort to match typical notebook FITS ops):
- Upload .fits or .fz files
- Select HDU / extension
- View header metadata (searchable)
- Display image with stretch options: linear, log, asinh, sqrt
- Auto vmin/vmax with adjustable percentiles
- Histogram and basic statistics (min, max, mean, median, std)
- Background estimation (median or sigma-clipped mean) and subtraction
- Smoothing (Gaussian) and median filter
- Basic cropping / ROI selection
- Save processed image as FITS and download processed PNG
- Option to show WCS overlay (if WCS present) -- uses astropy.wcs to print info; plotting overlay simple

Notes:
- This app is written to be run with Streamlit.
- Install requirements: pip install streamlit astropy numpy scipy matplotlib pillow

Usage: streamlit run streamlit_fits_uploader_and_processor.py
"""

import io
import os
import tempfile
from typing import Tuple

import numpy as np
import streamlit as st
from astropy.io import fits
from astropy.visualization import (AsinhStretch, LogStretch, PercentileInterval,
                                   ImageNormalize, LinearStretch)
from astropy.stats import sigma_clipped_stats
from scipy.ndimage import gaussian_filter, median_filter
import matplotlib.pyplot as plt
from PIL import Image

st.set_page_config(page_title="FITS Uploader & Processor", layout="wide")

# --- Helper functions -------------------------------------------------

def read_fits(file_obj) -> fits.HDUList:
    """Read an uploaded FITS file (file-like object) and return HDUList."""
    try:
        # astropy can read file-like objects
        hdul = fits.open(file_obj)
        return hdul
    except Exception as e:
        st.error(f"Failed to read FITS file: {e}")
        return None


def get_image_data(hdu) -> Tuple[np.ndarray, dict]:
    """Return 2D image array from an HDU if possible, and header dict."""
    hdr = dict(hdu.header)
    data = hdu.data
    # Some FITS store data in (N,1,M) or other shapes. Try to extract 2D.
    if data is None:
        return None, hdr
    data = np.array(data)
    # If more than 2 dims, try to select first slice with non-empty data
    if data.ndim == 3:
        # choose the first plane that's not all zeros
        for i in range(data.shape[0]):
            plane = data[i]
            if np.any(np.isfinite(plane)) and np.nanmax(np.abs(plane)) != 0:
                data = plane
                break
        else:
            data = data[0]
    if data.ndim > 2:
        # collapse remaining dims
        data = data.reshape(data.shape[0], -1)
    # convert to float
    try:
        data = data.astype(float)
    except Exception:
        data = np.array(data, dtype=float)
    return data, hdr


def apply_stretch(data: np.ndarray, stretch_name: str, percentiles=(1,99)) -> Tuple[np.ndarray, ImageNormalize]:
    """Return stretched image for display and the normalizer used."""
    # compute interval
    lo, hi = np.nanpercentile(data, percentiles)
    interval = PercentileInterval(percentiles[1]) if False else PercentileInterval(percentiles[1])
    # Use vmin/vmax by percentiles
    vmin, vmax = np.nanpercentile(data, percentiles)

    if stretch_name == "linear":
        stretch = LinearStretch()
    elif stretch_name == "log":
        stretch = LogStretch()  # maps negative -> nan internally
    elif stretch_name == "asinh":
        stretch = AsinhStretch()
    elif stretch_name == "sqrt":
        # sqrt is approximated by power stretch of 0.5 via normalize transform; use asinh for safety
        stretch = AsinhStretch()  # fallback
    else:
        stretch = LinearStretch()

    norm = ImageNormalize(vmin=vmin, vmax=vmax, stretch=stretch)
    # create displayed image
    with np.errstate(invalid='ignore'):
        normed = norm(data)
    return normed, norm


def compute_stats(data: np.ndarray):
    """Return simple statistics and sigma-clipped stats."""
    mean = np.nanmean(data)
    median = np.nanmedian(data)
    std = np.nanstd(data)
    minv = np.nanmin(data)
    maxv = np.nanmax(data)
    try:
        clip_mean, clip_med, clip_std = sigma_clipped_stats(data, sigma=3.0)
    except Exception:
        clip_mean, clip_med, clip_std = (np.nan, np.nan, np.nan)
    return {
        'min': float(minv), 'max': float(maxv), 'mean': float(mean), 'median': float(median), 'std': float(std),
        'clip_mean': float(clip_mean), 'clip_median': float(clip_med), 'clip_std': float(clip_std)
    }


def plot_image(data: np.ndarray, norm=None, title: str = None, cmap='gray'):
    fig, ax = plt.subplots(figsize=(6, 6))
    if norm is not None:
        ax.imshow(data, origin='lower', norm=norm, cmap=cmap)
    else:
        ax.imshow(data, origin='lower', cmap=cmap)
    ax.set_xticks([])
    ax.set_yticks([])
    if title:
        ax.set_title(title)
    plt.tight_layout()
    return fig


def save_array_to_fits(array: np.ndarray, header: dict = None) -> bytes:
    hdu = fits.PrimaryHDU(array)
    if header:
        for k, v in header.items():
            try:
                hdu.header[k] = v
            except Exception:
                # skip non-standard header values
                pass
    hdul = fits.HDUList([hdu])
    bio = io.BytesIO()
    hdul.writeto(bio)
    bio.seek(0)
    return bio.read()


def array_to_png_bytes(array: np.ndarray, norm=None) -> bytes:
    fig = plot_image(array, norm=norm)
    buf = io.BytesIO()
    fig.savefig(buf, bbox_inches='tight', dpi=150)
    plt.close(fig)
    buf.seek(0)
    return buf.read()


# --- Streamlit layout -------------------------------------------------

st.title("🔭 FITS Uploader & Processor")
st.markdown(
    "Upload a FITS file, inspect headers, view the image, apply common preprocessing (background subtraction, smoothing), and download results."
)

col1, col2 = st.columns([1, 2])

with col1:
    uploaded = st.file_uploader("Upload a FITS (.fits, .fz) file", type=['fits', 'fit', 'fz'], accept_multiple_files=False)
    st.markdown("---")
    st.markdown("**Display options**")
    stretch = st.selectbox("Stretch", ['linear', 'log', 'asinh', 'sqrt'])
    pmin = st.slider("Lower percentile (vmin)", 0, 50, 1)
    pmax = st.slider("Upper percentile (vmax)", 50, 100, 99)
    cmap = st.selectbox("Colormap", ['gray', 'viridis', 'inferno', 'magma', 'cividis'])
    show_hist = st.checkbox("Show histogram", value=True)
    show_header = st.checkbox("Show header", value=True)
    do_bg_sub = st.checkbox("Background subtraction", value=False)
    bg_method = st.selectbox("Background method", ['median', 'sigma_clipped'])
    apply_smooth = st.checkbox("Apply smoothing", value=False)
    sigma = st.slider("Gaussian sigma", 0.0, 10.0, 1.0)
    apply_median = st.checkbox("Apply median filter", value=False)
    median_size = st.slider("Median filter size", 1, 11, 3)
    do_crop = st.checkbox("Crop (select ROI)", False)
    download_processed_name = st.text_input("Processed FITS filename", "processed.fits")

with col2:
    if uploaded is None:
        st.info("Upload a FITS file to get started. You can test with astronomical FITS files (e.g., sample from HST, SDSS).")
        st.stop()

    # Read fits
    hdul = read_fits(uploaded)
    if hdul is None:
        st.stop()

    # List HDUs
    hdu_options = [f"{i}: {hdu.name}" for i, hdu in enumerate(hdul)]
    selection_idx = st.selectbox("Select HDU", options=list(range(len(hdu_options))), format_func=lambda i: hdu_options[i])
    hdu = hdul[selection_idx]

    data, header = get_image_data(hdu)
    if data is None:
        st.error("Selected HDU has no image data.")
        st.stop()

    st.subheader("Header & Metadata")
    if show_header:
        # display a searchable header
        st.json({k: str(v) for k, v in header.items()})

    # compute stats
    stats = compute_stats(data)
    st.write("**Image statistics (raw)**")
    st.write(stats)

    # Background subtraction
    processed = data.copy()
    if do_bg_sub:
        if bg_method == 'median':
            bg = np.nanmedian(processed)
        else:
            # sigma clipped mean
            clip_mean, clip_med, clip_std = sigma_clipped_stats(processed, sigma=3.0)
            bg = clip_med if not np.isnan(clip_med) else clip_mean
        processed = processed - bg
        st.write(f"Background ({bg_method}) estimated = {float(bg):.3e}")

    # Smoothing
    if apply_smooth and sigma > 0:
        processed = gaussian_filter(processed, sigma=sigma)

    if apply_median and median_size > 1:
        processed = median_filter(processed, size=median_size)

    # Cropping
    if do_crop:
        st.markdown("**Crop settings (pixels)**")
        ny, nx = processed.shape
        colc1, colc2, colc3 = st.columns([1, 1, 1])
        with colc1:
            x0 = st.number_input("x0 (left)", min_value=0, max_value=nx - 1, value=0)
            x1 = st.number_input("x1 (right)", min_value=0, max_value=nx - 1, value=nx - 1)
        with colc2:
            y0 = st.number_input("y0 (bottom)", min_value=0, max_value=ny - 1, value=0)
            y1 = st.number_input("y1 (top)", min_value=0, max_value=ny - 1, value=ny - 1)
        # clamp
        x0, x1 = int(x0), int(x1)
        y0, y1 = int(y0), int(y1)
        if x1 <= x0 or y1 <= y0:
            st.warning("Invalid crop box; showing full image")
        else:
            processed = processed[y0:y1, x0:x1]

    # Stretch & normalize for display
    normed, norm = apply_stretch(processed, stretch_name=stretch, percentiles=(pmin, pmax))

    # Main image display
    st.subheader("Image preview")
    fig = plot_image(processed, norm=norm, title=f"HDU {selection_idx}: {hdul[selection_idx].name}", cmap=cmap)
    st.pyplot(fig)

    # Histogram
    if show_hist:
        fig2, ax2 = plt.subplots(figsize=(6, 3))
        ax2.hist(processed.flatten()[~np.isnan(processed.flatten())], bins=200)
        ax2.set_title("Pixel value distribution")
        ax2.set_yscale('linear')
        st.pyplot(fig2)

    # Download processed FITS
    st.markdown("---")
    st.subheader("Export")
    fits_bytes = save_array_to_fits(processed, header=header)
    st.download_button("Download processed FITS", data=fits_bytes, file_name=download_processed_name, mime='application/fits')

    # Download preview PNG
    png_bytes = array_to_png_bytes(processed, norm=norm)
    st.download_button("Download preview PNG", data=png_bytes, file_name='preview.png', mime='image/png')

    st.success("Processing complete.")

# End of app


Overwriting app.py
